In [2]:
import pandas
import sqlite3

In [3]:
connection = sqlite3.connect('nearby_stars.db')
c = connection.cursor()

## Add full MH08 age table to database

In [8]:
age_tab = pandas.read_csv('mh08_agetable.csv')
age_tab.to_sql('mh08_agetable', connection)

In [9]:
c.execute('PRAGMA table_info(mh08_agetable);')
c.fetchall()

[(0, u'index', u'INTEGER', 0, None, 0),
 (1, u'HD', u'INTEGER', 0, None, 0),
 (2, u'GJ', u'TEXT', 0, None, 0),
 (3, u'OName', u'TEXT', 0, None, 0),
 (4, u'l_tau1', u'TEXT', 0, None, 0),
 (5, u'tau1', u'REAL', 0, None, 0),
 (6, u'l_tau2', u'TEXT', 0, None, 0),
 (7, u'tau2', u'REAL', 0, None, 0)]

## Create ages table

In [4]:
#c.execute('DROP TABLE ages')

In [5]:
create_table = '''
CREATE TABLE IF NOT EXISTS ages (
sid INTEGER PRIMARY KEY,
age REAL,
age_err REAL,
bibcode TEXT
)
'''

In [6]:
c.execute(create_table)

## Create subquery connecting star in MH08 table to star in main simbad_FGK_nobinaries table

In [42]:
cmd = '''
SELECT stars_FGK_nobinaries.sid, mh08_agetable.OName, stars_FGK_nobinaries.main_id, 
'HD ' || mh08_agetable.hd, stars_FGK_nobinaries.hd, mh08_agetable.gj, LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ'))
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(0, u'epsilon Eri', u'* eps Eri', u'HD 22049', u'HD  22049', u'144', u'144')
(1, u'tau Cet', u'* tau Cet', u'HD 10700', u'HD  10700', u'71', u'71')
(2, u'82 Eri', u'* e Eri', u'HD 20794', u'HD  20794', u'139', u'139')
(3, u'delta Pav', u'* del Pav', u'HD 190248', u'HD 190248', u'780', u'780')
(8, u'61 Vir', u'*  61 Vir', u'HD 115617', u'HD 115617', u'506', u'506')
(9, u'zeta Tuc', u'* zet Tuc', u'HD 1581', u'HD   1581', u'17', u'17')
(10, u'HR 7722', u'HD 192310', u'HD 192310', u'HD 192310', u'785', u'785')
(13, u'CF UMa', u'HD 103095', u'HD 103095', u'HD 103095', u'451A', u'451')
(18, u'alpha Men', u'* alf Men', u'HD 43834', u'HD  43834', u'231', u'231')
(20, u'HR 6806', u'HD 166620', u'HD 166620', u'HD 166620', u'706', u'706')
(21, u'54 Psc', u'*  54 Psc', u'HD 3651', u'HD   3651', u'27A', u'27')
(27, u'HR 3384', u'HD  72673', u'HD 72673', u'HD  72673', u'309', u'309')
(30, u'HR 3259', u'HD  69830', u'HD 69830', u'HD  69830', u'302', u'302')
(36, u'HR 683', u'HD  14412', u'HD 1441

39 is the same number of stars I found when querying Simbad with the MH08 sample list and eliminating binaries in the same way I did to create stars_FGK_nobinaries, so this cross match catches all of them!

Now check query to get the columns that I actually want to put in the ages table.

NOTES
- Sec 4.4 states that tau2 are the preferred ages and that the conservative error is 50%, so I will directly put the error in now.
- Found bibcode using ADS: 2008ApJ...687.1264M

In [44]:
cmd = '''
SELECT stars_FGK_nobinaries.sid, mh08_agetable.tau2, 0.5*mh08_agetable.tau2, '2008ApJ...687.1264M'
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(0, 0.8, 0.4, u'2008ApJ...687.1264M')
(1, 5.8, 2.9, u'2008ApJ...687.1264M')
(2, 6.1, 3.05, u'2008ApJ...687.1264M')
(3, 6.9, 3.45, u'2008ApJ...687.1264M')
(8, 6.1, 3.05, u'2008ApJ...687.1264M')
(9, 2.1, 1.05, u'2008ApJ...687.1264M')
(10, 8.9, 4.45, u'2008ApJ...687.1264M')
(13, 5.3, 2.65, u'2008ApJ...687.1264M')
(18, 5.5, 2.75, u'2008ApJ...687.1264M')
(20, 7.1, 3.55, u'2008ApJ...687.1264M')
(21, 7.7, 3.85, u'2008ApJ...687.1264M')
(27, 6.5, 3.25, u'2008ApJ...687.1264M')
(30, 6.1, 3.05, u'2008ApJ...687.1264M')
(36, 4.3, 2.15, u'2008ApJ...687.1264M')
(35, 5.9, 2.95, u'2008ApJ...687.1264M')
(31, 5.9, 2.95, u'2008ApJ...687.1264M')
(40, 2.6, 1.3, u'2008ApJ...687.1264M')
(42, 0.6, 0.3, u'2008ApJ...687.1264M')
(49, 4.8, 2.4, u'2008ApJ...687.1264M')
(53, 4.2, 2.1, u'2008ApJ...687.1264M')
(48, 8.8, 4.4, u'2008ApJ...687.1264M')
(50, 4.4, 2.2, u'2008ApJ...687.1264M')
(60, 0.2, 0.1, u'2008ApJ...687.1264M')
(72, 6.8, 3.4, u'2008ApJ...687.1264M')
(66, 3.6, 1.8, u'2008ApJ...687.1264M')
(63, 2.5, 1.25

## Add rows to the ages table

In [46]:
cmd = '''
INSERT INTO ages 
SELECT stars_FGK_nobinaries.sid, mh08_agetable.tau2, 0.5*mh08_agetable.tau2, '2008ApJ...687.1264M'
FROM mh08_agetable
    INNER JOIN stars_FGK_nobinaries 
    ON LTRIM(LTRIM(stars_FGK_nobinaries.hd,'HD')) LIKE mh08_agetable.hd
    OR LTRIM(LTRIM(stars_FGK_nobinaries.GJ,'GJ')) LIKE mh08_agetable.gj
'''
c.execute(cmd)

Check result!

In [47]:
cmd = '''
SELECT * FROM ages
'''
c.execute(cmd)
c.fetchall()

[(0, 0.8, 0.4, u'2008ApJ...687.1264M'),
 (1, 5.8, 2.9, u'2008ApJ...687.1264M'),
 (2, 6.1, 3.05, u'2008ApJ...687.1264M'),
 (3, 6.9, 3.45, u'2008ApJ...687.1264M'),
 (8, 6.1, 3.05, u'2008ApJ...687.1264M'),
 (9, 2.1, 1.05, u'2008ApJ...687.1264M'),
 (10, 8.9, 4.45, u'2008ApJ...687.1264M'),
 (13, 5.3, 2.65, u'2008ApJ...687.1264M'),
 (18, 5.5, 2.75, u'2008ApJ...687.1264M'),
 (20, 7.1, 3.55, u'2008ApJ...687.1264M'),
 (21, 7.7, 3.85, u'2008ApJ...687.1264M'),
 (27, 6.5, 3.25, u'2008ApJ...687.1264M'),
 (30, 6.1, 3.05, u'2008ApJ...687.1264M'),
 (31, 5.9, 2.95, u'2008ApJ...687.1264M'),
 (35, 5.9, 2.95, u'2008ApJ...687.1264M'),
 (36, 4.3, 2.15, u'2008ApJ...687.1264M'),
 (40, 2.6, 1.3, u'2008ApJ...687.1264M'),
 (42, 0.6, 0.3, u'2008ApJ...687.1264M'),
 (48, 8.8, 4.4, u'2008ApJ...687.1264M'),
 (49, 4.8, 2.4, u'2008ApJ...687.1264M'),
 (50, 4.4, 2.2, u'2008ApJ...687.1264M'),
 (53, 4.2, 2.1, u'2008ApJ...687.1264M'),
 (60, 0.2, 0.1, u'2008ApJ...687.1264M'),
 (63, 2.5, 1.25, u'2008ApJ...687.1264M'),
 (64, 5

In [51]:
cmd = '''
SELECT * FROM ages
INNER JOIN stars_FGK_nobinaries ON ages.sid = stars_FGK_nobinaries.sid
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
for row in rows:
    print row

39
(0, 0.8, 0.4, u'2008ApJ...687.1264M', 0, u'* eps Eri', u'53.2326873475', u'-9.45825865694444', u'HD  22049', u'GJ   144', u'WDS J03329-0927A,WDS J03329-0927Aa,Ab', u'K2V', u'1989ApJS...71..245K', 310.94, 0.16, u'2007A&A...474..653V', 3.73, None, u'2002yCat.2237....0D', 3.3691, 0.004, u'2018yCat.1345....0G')
(1, 5.8, 2.9, u'2008ApJ...687.1264M', 1, u'* tau Cet', u'26.0170142608333', u'-15.9374795977778', u'HD  10700', u'GJ    71', u'WDS J01441-1556A', u'G8V', u'1989ApJS...71..245K', 273.96, 0.17, u'2007A&A...474..653V', 3.5, None, u'2002yCat.2237....0D', 3.1336, 0.0075, u'2018yCat.1345....0G')
(2, 6.1, 3.05, u'2008ApJ...687.1264M', 2, u'* e Eri', u'49.9818788966667', u'-43.0697826375', u'HD  20794', u'GJ   139', None, u'G6V', u'1989ApJS...71..245K', 165.47, 0.19, u'2007A&A...474..653V', 4.27, None, u'2002yCat.2237....0D', 3.9814, 0.0037, u'2018yCat.1345....0G')
(3, 6.9, 3.45, u'2008ApJ...687.1264M', 3, u'* del Pav', u'302.18170612375', u'-66.1820675763889', u'HD 190248', u'GJ   780',

In [52]:
connection.commit()

In [53]:
connection.close()